# EXPLOTATION ZONE

1. Load data
2. Integrate data
3. Perform concrete data quality 

## 1. Load data from trusted zone


In [ ]:
!pip install duckdb

import duckdb
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.4 MB 4.2 MB/s 


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#con = duckdb.connect(database='/content/drive/MyDrive/projecteADSDB/explotation/explotation.duckdb', read_only=False)

con = duckdb.connect(database='./explotation/explotation.duckdb', read_only=False)


df_governance = pd.read_sql("SELECT * FROM Governance_Data", con)
df_olympics = pd.read_sql("SELECT * FROM Olympics_Data", con)
df_countrylevel = pd.read_sql("SELECT * FROM Country_Level_Data", con)

In [ ]:
df_governance.head()

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],2000 [YR2000],2004 [YR2004],2008 [YR2008],2012 [YR2012],2016 [YR2016],2020 [YR2020]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.271724,-1.350647,-1.672096,-1.430373,-1.534049,-1.480538
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2.000000,2.000000,5.000000,7.000000,10.000000,10.000000,9.000000
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,4.787234,6.403941,0.485437,1.421801,3.846154,4.807693
3,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,27.419355,30.851065,15.270936,4.368932,11.848341,9.615385,10.576923


In [ ]:
df_olympics.head()

,year,city,discipline,sport_group,ioc_code,iso_code,country,region,event_sex,participant_type,event,medal,olympic_type
0,1992,Barcelona,Diving,Swimming and Diving,CHN,CHN,China,Asia,Men,Single,10M Platform,Bronze,summer
1,1992,Barcelona,Diving,Swimming and Diving,USA,USA,USA,Americas,Women,Single,10M Platform,Bronze,summer
2,1992,Barcelona,Diving,Swimming and Diving,EUN,EUN,Unified Team,noregion,Men,Single,3M Springboard,Bronze,summer
3,1992,Barcelona,Diving,Swimming and Diving,GER,DEU,Germany,Europe,Women,Single,3M Springboard,Bronze,summer
4,1992,Barcelona,Swimming,Swimming and Diving,USA,USA,USA,Americas,Men,Single,100M Backstroke,Bronze,summer


In [ ]:
df_countrylevel.head()

,year,city,iso,country,region,olympic_type,population,GDP_constant_2010_USD,land_area_km_sq,notes,population_%,GDP_%,land_area_%
0,1992,Barcelona,ALB,Albania,Europe,summer,3247039.0,4.183027e+09,27400,None,0.000614,0.000110,0.000248
1,1992,Barcelona,DZA,Algeria,Africa,summer,27181094.0,9.252182e+10,2381740,None,0.005142,0.002430,0.021531
2,1992,Barcelona,ASM,American Samoa,Oceania,summer,49593.0,3.762957e+08,200,None,0.000009,0.000010,0.000002
3,1992,Barcelona,AND,Andorra,Europe,summer,58888.0,2.018601e+09,470,None,0.000011,0.000053,0.000004
4,1992,Barcelona,AGO,Angola,Africa,summer,12968345.0,2.945149e+10,1246700,None,0.002453,0.000773,0.011270


## 2.2. Integrate with data governance


## 2.2.2. integrate dataframes

In [ ]:
df_countrylevel['year'] = df_countrylevel['year'].astype(int)
df_governance['year'] = df_governance['year'].astype(int)

data = pd.merge(df_countrylevel, df_governance, left_on=['year', 'iso'], right_on=['year', 'Country Code'])
data.drop(columns=['Country Code'], inplace=True)

In [ ]:
data.drop(columns=['notes', 'population_%', 'GDP_%', 'land_area_%'], inplace=True)

,year,city,iso,country,region,olympic_type,population,GDP_constant_2010_USD,land_area_km_sq,CC,G,PV,RL,RQ,VA
0,1996,Atlanta,AFG,Afghanistan,Asia,summer,17822884.0,6.641568e+09,652860,-1.291705,-2.175167,-2.417310,-1.788075,-2.090330,-1.908540
1,1996,Atlanta,ALB,Albania,Europe,summer,3168033.0,6.137399e+09,27400,-0.893903,-0.688588,-0.336625,-0.684482,-0.474402,-0.648298
2,1996,Atlanta,DZA,Algeria,Africa,summer,29411415.0,9.699483e+10,2381740,-0.566741,-1.088766,-1.783311,-1.218195,-0.907437,-1.166290
3,1996,Atlanta,AND,Andorra,Europe,summer,64360.0,2.199520e+09,470,1.318143,1.414038,1.169522,1.158363,1.266440,1.563217
4,1996,Atlanta,AGO,Angola,Africa,summer,14682284.0,2.817841e+10,1246700,-1.167702,-0.859940,-2.061917,-1.630469,-1.415345,-1.578164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,2016,Rio,SMR,San Marino,Europe,summer,33203.0,1.746352e+09,60,NaN,NaN,NaN,NaN,NaN,1.191460
1160,2016,Rio,BMU,Bermuda,Americas,summer,65331.0,5.274855e+09,50,1.223311,1.323129,0.999670,0.858730,0.867860,NaN
1161,2016,Rio,TUV,Tuvalu,Oceania,summer,11097.0,3.776000e+07,30,NaN,NaN,NaN,0.595924,NaN,1.081802
1162,2016,Rio,NRU,Nauru,Oceania,summer,13049.0,1.259303e+08,20,NaN,NaN,NaN,-0.776703,NaN,0.507922


# Store data

In [ ]:
con.execute("DROP TABLE IF EXISTS Governance_Data;")
con.execute("DROP TABLE IF EXISTS Olympics_Data;")
con.execute("DROP TABLE IF EXISTS Country_Level_Data;")

con.execute("DROP TABLE IF EXISTS Olympics_Data;")
con.execute("CREATE TABLE IF NOT EXISTS Olympics_Data AS SELECT * FROM df_olympics")

con.execute("DROP TABLE IF EXISTS Country_Data")
con.execute("CREATE TABLE IF NOT EXISTS Country_Data AS SELECT * FROM data")

con.execute("SHOW TABLES").fetchall()

[('Country_Data',), ('Olympics_Data',)]

In [ ]:
con.close()